<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Обучение</a></span></li></ul></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span><ul class="toc-item"><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Обучение</a></span></li></ul></li><li><span><a href="#Тест" data-toc-modified-id="Тест-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тест</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Обучим модель классифицировать комментарии на позитивные и негативные.

## Подготовка

In [1]:
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import re
import spacy

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score

Загрузим необходимые данные.

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

Посмотрим на загружаемые данные.

In [3]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


В данных имеется неописанный столбец 'Unnamed: 0', мы можем удалить его, поскольку для обучения моделей он нам не понадобится.

In [4]:
data = data.drop('Unnamed: 0', axis=1)

Из полной информации по данным следует, что в них нет явных пропусков и повторов.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Также можно заметить, что позитивных коментариев на порядок больше, чем негативных.

In [6]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

---

Лемматизируем и очистим от ненужных символов тексты для облегчения их последующей обработки.

In [7]:
# возвращает очищенный от лишних символов текст
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z\s]', ' ', text).split())

# возвращает массив с леммами из массива с текстами
def lemmatize(txt):
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    l = []
    for i in tqdm(txt):
        doc = nlp(clear_text(i))
        l.append(' '.join([token.lemma_ for token in doc]))
    return l

st = time.time()
data['text'] = lemmatize(data['text']) # в дальнейшем тексты нам не понадобятся
te = time.time()
print(te - st, 'с')

100%|██████████| 159292/159292 [25:17<00:00, 104.94it/s]

1518.7609388828278 с


## Обучение

<div class="alert alert-info">
<b>Комментарий:</b>
Насколько я могу понять, с точки зрения точности лучше всего использовать BERT, но поскольку мой компьютер, как и сам Jupyter имеет некоторые существенные ограничения на оперативную память, я вынужден использовать другой подход.
</div>

Отделим целевые признаки от нецелевых и разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношение 5:2:3 (50%:20%:30%).

<div class="alert alert-info">
<b>Комментарий:</b>
Было бы замечательно использовать для исследования стандартное разбиение данных (3:1:1), но я не могу это сделать без того, чтобы Jupyter в очередной раз не убил ядро. Методом научного тыка было установлено, что для обучения я могу использовать только 50% имеющихся данных.
</div>

In [9]:
x = data.drop('toxic', axis=1)
y = data['toxic']
del data # экономим ресурсы


random_state = 42 # random_state

# разбиение признаков
x_train, x_valid, x_test = \
              np.split(x.sample(frac=1, random_state=random_state), 
                       [int(.5*len(x)), int(.7*len(x))])

# разбиение целевого признака
y_train, y_valid, y_test = \
              np.split(y.sample(frac=1, random_state=random_state), 
                       [int(.5*len(y)), int(.7*len(y))])

Для определения характера текстов применим величины TF-IDF как признаки. Для этого удалим лишние слова из рассматриваемых выборок и расчитаем TF-IDF.

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
v = TfidfVectorizer(stop_words=stopwords)

x_train['text'] = x_train['text']
x_valid['text'] = x_valid['text']
x_test['text'] = x_test['text']

x_train = v.fit_transform(x_train['text'])
x_valid = v.transform(x_valid['text'])
x_test = v.transform(x_test['text'])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Логистическая регрессия

Обучим модель логистической регресси на указанных данных.

#### Обучение

In [11]:
f0 = 0
st = time.time()
for (i,j) in [(i,j) for i in range(100,201,50) for j in np.arange(3.2,3.7,0.4)]:  # LogisticRegression     # F1: ~0.75, быстро
#for (i,j) in [(i,j) for i in range(200,221,10) for j in range(40,43)]:           # RandomForestClassifier # F1: отвратительный
#for (i,j) in [(i,j) for i in range(2,4) for j in np.arange(0.6,1.5,0.4)]:        # SVC                    # F1: очень долго (но ядро живёт)
#for (i,j) in [(i,j) for i in range(3,5) for j in range(1,3)]:                    # DecisionTreeClassifier # F1: ~0.71, долго
#for (i,j) in [(i,j) for i in range(3,6) for j in range(20,41,10)]:               # KNeighborsClassifier   # F1: отвратительный, ядро умирает
    te = time.time()
    print(i,j, ' : ', te - st, 'c')
    model = LogisticRegression(random_state=random_state, max_iter=i, C=j)
#    model = RandomForestClassifier(
#        random_state=random_state,
#        n_estimators=i,                      # количество деревьев
#        max_depth=j,                         # максимальная глубина
#        class_weight='balanced'              # обучаем модель с взвешаннами классами
#    )
#    model = SVC(random_state=random_state, degree=i, C=j)
#    model = DecisionTreeClassifier(random_state=random_state, min_samples_split=i, min_samples_leaf=j)
#    model = KNeighborsClassifier(n_neighbors=i, leaf_size=j)
    model.fit(x_train,y_train)
    predict = model.predict(x_valid)
    ff = f1_score(predict,y_valid)
    print('f1:', ff, '\n')
    if (ff > f0):
        f0 = ff
        ii = i
        jj = j
        best_model_LogisticRegression = model

print('F1 лучшей модели:', f0)

100 3.2  :  0.0002734661102294922 c


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


f1: 0.7623462292743804 

100 3.6  :  42.921725273132324 c


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


f1: 0.7653532126375577 

150 3.2  :  88.60000777244568 c
f1: 0.7623462292743804 

150 3.6  :  152.2264449596405 c
f1: 0.7648624667258208 

200 3.2  :  209.88601231575012 c
f1: 0.7623462292743804 

200 3.6  :  272.5654413700104 c
f1: 0.7648624667258208 

F1 лучшей модели: 0.7653532126375577


### Дерево решений

Кроме того обучим модель дерева решений.

#### Обучение

In [ ]:
f0 = 0
st = time.time()
#for (i,j) in [(i,j) for i in range(100,201,50) for j in np.arange(3.2,3.7,0.4)]: # LogisticRegression     # F1: ~0.75, быстро
#for (i,j) in [(i,j) for i in range(200,221,10) for j in range(40,43)]:           # RandomForestClassifier # F1: отвратительный
#for (i,j) in [(i,j) for i in range(2,4) for j in np.arange(0.6,1.5,0.4)]:        # SVC                    # F1: очень долго (но ядро живёт)
for (i,j) in [(i,j) for i in range(3,5) for j in range(1,3)]:                     # DecisionTreeClassifier # F1: ~0.71, долго
#for (i,j) in [(i,j) for i in range(3,6) for j in range(20,41,10)]:               # KNeighborsClassifier   # F1: отвратительный, ядро умирает
    te = time.time()
    print(i,j, ' : ', te - st, 'c')
#    model = LogisticRegression(random_state=random_state, max_iter=i, C=j)
#    m = RandomForestClassifier(
#        random_state=random_state,
#        n_estimators=i,                      # количество деревьев
#        max_depth=j,                         # максимальная глубина
#        class_weight='balanced'              # обучаем модель с взвешаннами классами
#    )
#    model = SVC(random_state=random_state, degree=i, C=j)
    model = DecisionTreeClassifier(random_state=random_state, min_samples_split=i, min_samples_leaf=j)
#    model = KNeighborsClassifier(n_neighbors=i, leaf_size=j)
    model.fit(x_train,y_train)
    predict = model.predict(x_valid)
    ff = f1_score(predict,y_valid)
    print('f1:', ff, '\n')
    if (ff > f0):
        f0 = ff
        ii = i
        jj = j
        best_model_DecisionTreeClassifier = model

print('F1 лучшей модели:', f0)

3 1  :  0.00014328956604003906 c
f1: 0.7114880493446416 

3 2  :  191.99816513061523 c
f1: 0.6957642132388469 

4 1  :  346.26735258102417 c


### Тест

Моделью с наилучшим показателем F1 оказалась логиситческая регрессия.

In [15]:
predict = best_model_LogisticRegression.predict(x_test)
f1_score(predict,y_test)

0.7615557394491075

## Выводы

В данной работе была получена модель, позволяющая классифицировать позитивные и негативные коментарии. Параметр F1, полученный на тестовых данных укладывается в допустимое значение. Данная модель поможет упростить поиск негативных коментариев для интернет магазина «Викишоп».